In [ ]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv
from azure.ai.evaluation import SimilarityEvaluator

backend_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', 'backend'))
sys.path.append(backend_path)

from genai_vanilla_agents.workflow import Workflow
from genai_vanilla_agents.conversation import Conversation

from agents.fsi_insurance.group_chat import create_group_chat_insurance

load_dotenv()

parent_dir = os.path.abspath('..')

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
# Initialize Azure OpenAI Connection with your environment variables
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_MODEL"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

similarity = SimilarityEvaluator(model_config)

In [ ]:
def get_agentic_response(row):
    """
    * pulled from MONETA.
    * removed conversation history
    * added similarity score
    """
    
    query = row['Queries']
    ground_truth = row['Answer']
    
    conversation_history = Conversation(messages=[], variables={})
    team = create_group_chat_insurance()
    
    workflow = Workflow(askable=team, conversation=conversation_history)
    workflow.run(query)
    
    delta = len(workflow.conversation.messages) 
    
    first_column_name = row.index[0]
    first_field = row[first_column_name]
    print(f"Record : {first_field} - {len(workflow.conversation.messages)}")
    
    new_messages = workflow.conversation.messages[-delta:]
    
    assistant_message = None
    
    for message in reversed(new_messages):
        if message.get('role') == 'assistant':
            assistant_message = message
            break
        
    if assistant_message:
        content = assistant_message.get('content', '')
        name = assistant_message.get('name', '')
    else:
        message = next(reversed(new_messages))
        content = f"{message.get('content', ' NO CONTENT FOUND ')} - {message.get('name', ' NO NAME FOUND ')}"
        name = 'ERROR'
        print(f"ERROR: {content}")
        
    # Scoring
    similarity_score = similarity(query=query, response=content, ground_truth=ground_truth)
    
     # Add a delay (e.g., 5 second) between requests
    # time.sleep(3)

    # Return the content and name as a Series
    return pd.Series({'agentic_answer': content, 'agent_name': name, 'similarity_score': similarity_score.get('gpt_similarity')})


In [ ]:
df = pd.read_csv('../../../../travel_question_bank_answered_anon.csv')

In [ ]:
# Single row 
row = df.iloc[1]

print(f"ROW:\n\n{row}\n\n")

dd = get_agentic_response(row)

print(f"Agent name: {dd['agent_name']}")
print(f"Similarity score: {dd['similarity_score']}")
print(f"Agentic answer: {dd['agentic_answer']}")
print(f"---------------------------")
print(f"Ground Truth Answer: {row['Answer']}")


In [ ]:
# Apply the function to each row
df[['agentic_answer', 'agent_name', 'similarity_score']] = df.apply(get_agentic_response, axis=1)

In [7]:
df.groupby('agent_name')['similarity_score'].mean().sort_values(ascending=False)

agent_name
Product    3.204082
CRM        2.000000
Planner    1.000000
Name: similarity_score, dtype: float64

In [ ]:
mean_similarity_score = df['similarity_score'].mean()
print(f"The mean similarity score is: {mean_similarity_score:.2f}")

The mean similarity score is: 2.85


In [ ]:

df.to_csv('agentic_responses_gbb.csv', index=False)